In [ ]:
import pandas_datareader.data as web
from pymatgen import Structure
from mpcontribs.client import load_client

Use your API key to set up the client.

In [ ]:
project = 'my_project'
client = load_client('your-api-key-here')

The *dir* command lists available resources and operations.

In [ ]:
print(dir(client))
print(dir(client.projects))

Use the *projects* resource to retrieve or update project information. The *_fields* argument controls which fields are included in the response.

In [ ]:
client.projects.get_entry(pk=project, _fields=['title', 'authors', 'description', 'urls']).result()

The *update_entry* operation returns a dictionary of updated fields.

In [ ]:
client.projects.update_entry(pk=project, project={
     'title': 'New Title', # 'urls': {'PRC': 'https://google.fr'}, 'urls': {'PRC': None}
}).result()

Use the *contributions* resource to create, retrieve, and update contributions.

In [ ]:
identifier = 'mp-1002'
cid = client.contributions.create_entries(contributions=[{
    'project': project, 'identifier': identifier,
    'data': {'E': '3.33 eV', 'E|V': {'a': 1, 'b': '3 cm'}}
}]).result()['data'][0]['id']

In [ ]:
client.contributions.get_entry(pk=cid, _fields=['id', 'identifier', 'data']).result()

In [ ]:
client.contributions.update_entry(pk=cid, contribution={
        'identifier': 'mp-1004', 'data': {'E': '14 eV'}   
}).result()

Use the *tables* resource to add tables to contributions.

In [ ]:
table = dict(contribution=cid, name='Dow-Jones Index')
data = web.DataReader('^DJI', 'stooq').to_dict('split')
data.pop('index')
table.update(data)

In [ ]:
tid = client.tables.create_entry(table=table.to_dict()).result()['id']

In [ ]:
entry = client.tables.get_entry(
    pk=tid, _fields=['id', 'columns', 'data'], data_per_page=3, data_page=5
).result()
Table.from_dict(entry)

In [ ]:
client.tables.update_entry(pk=tid, table={'name': 'DJI'}).result()

Use the *structures* resource to add structures to contributions.

In [ ]:
structure = Structure.from_file('Fe3O4.cif')
sdct = dict(contribution=cid, name='Fe3O4')
sdct.update(structure.as_dict())
sid = client.structures.create_entry(structure=sdct).result()['id']

In [ ]:
Structure.from_dict(client.structures.get_entry(
    pk=sid, _fields=['lattice', 'sites']
).result())

In [ ]:
client.structures.update_entry(pk=sid, structure={'name': 'Fe3 O4'}).result()

Finally, publish and/or delete the project or its components.

**TODO** use update_entries to also publish according contributions, tables, and structures

In [ ]:
client.structures.delete_entry(pk=sid).result()
client.tables.delete_entry(pk=tid).result()

deleting contributions (projects) also deletes corresponding structures, tables, notebooks and cards (contributions).

In [ ]:
client.contributions.delete_entry(pk=cid).result()  
# client.projects.delete_entry(pk=project).result()

**TODO** how to add a custom overview graph